In [ ]:
import tensorflow as tf
import mlflow
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
import sys
sys.executable

In [ ]:
!python --version

In [ ]:
print(tf.__version__)

In [ ]:
print(mlflow.__version__)

In [ ]:
def load_data():
    fashion_mnist = tf.keras.datasets.fashion_mnist
    (train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()
    
    class_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat',
               'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']
    
    return train_images, train_labels, test_images, test_labels, class_names

In [ ]:
train_images, train_labels, test_images, test_labels, class_names= load_data()

In [ ]:
print(train_images.shape)
#Each Label is between 0-9train_labels
print(test_images.shape)

In [ ]:
def get_val_data(test_images, test_labels):
    test_images_sub_arr = np.array_split(test_images, 2)
    val_images = test_images_sub_arr[0]
    test_images = test_images_sub_arr[1]
    

    test_labels_sub_arr = np.array_split(test_labels, 2)
    val_labels = test_labels_sub_arr[0]
    test_labels = test_labels_sub_arr[1]
    
    return val_images, val_labels, test_images, test_labels

In [ ]:
val_images, val_labels, test_images, test_labels = get_val_data(test_images, test_labels)

In [ ]:
print(val_images.shape)
print(test_images.shape)

print(val_labels.shape)
print(test_labels.shape)

### Preprocess the data

The data must be preprocessed before training the network.

**If you inspect the images in the training set, you will see that the pixel values fall in the range of 0 to 255.**

In [ ]:
def check_image_pixel_values(single_image):
    plt.figure()
    plt.imshow(single_image)
    plt.colorbar()
    plt.grid(False)
    plt.show()

In [ ]:
check_image_pixel_values(train_images[1])

### Image Scaling

Scale these values to a range of 0 to 1 before feeding them to the neural network model. 

To do so, divide the values by 255. It's important that the training set, validation set and testing set be preprocessed in the same way:

In [ ]:
def image_scaling(img_arr):
    scaled_img = img_arr / 255.0
    return scaled_img

In [ ]:
scaled_train_images = image_scaling(train_images)
scaled_val_images = image_scaling(val_images)
scaled_test_images = image_scaling(test_images)

In [ ]:
def verify_data_format(img_arr, img_labels,class_names):
    plt.figure(figsize=(10,10))
    for i in range(25):
        plt.subplot(5,5,i+1)
        plt.xticks([])
        plt.yticks([])
        plt.grid(False)
        plt.imshow(img_arr[i], cmap=plt.cm.binary)
        plt.xlabel(class_names[img_labels[i]])
    plt.show()

In [ ]:
verify_data_format(scaled_train_images, train_labels,class_names)

In [ ]:
verify_data_format(scaled_val_images, val_labels,class_names)

In [ ]:
verify_data_format(scaled_test_images, test_labels,class_names)

### Build the model

In [ ]:
def tf_model_build():
    model = tf.keras.Sequential([
        tf.keras.layers.Flatten(input_shape=(28, 28)),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dense(10)
    ])
    return model

### Compile the model

In [ ]:
def tf_compile_model(model):
    model.compile(optimizer='adam',
                  loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                  metrics=['accuracy'])
    return model

### Training the model

In [ ]:
import mlflow
mlflow.tensorflow.autolog(every_n_iter=2)

Note:: Autologging is known to be compatible with the following package versions: ``2.3.0`` <= ``tensorflow`` <= ``2.12.0``. Autologging may not succeed when used with package versions outside of this range.

In [ ]:
def tf_train_model(scaled_train_images, train_labels, epochs, batch_size, scaled_val_images, val_labels):
    model.fit(scaled_train_images, 
              train_labels, 
              epochs=epochs, 
              batch_size=batch_size, 
              validation_data=(scaled_val_images, val_labels)
             )
    return model

In [ ]:
model = tf_model_build()
model = tf_compile_model(model)

epochs = 10
batch_size = 32
model = tf_train_model(scaled_train_images, train_labels, epochs, batch_size, scaled_val_images, val_labels)


### Evaluate Model in test Images

In [ ]:
train_loss, train_acc = model.evaluate(scaled_train_images,  train_labels, verbose=2)
print('\ntrain accuracy:', train_acc)  

In [ ]:
val_loss, val_acc = model.evaluate(scaled_val_images,  val_labels, verbose=2)
print('\nval accuracy:', val_acc)  

In [ ]:
test_loss, test_acc = model.evaluate(test_images,  test_labels, verbose=2)
print('\nTest accuracy:', test_acc)  

In [ ]:
predictions = model.predict(test_images)
predictions[0]

In [ ]:
import numpy as np
np.argmax(predictions[0])
#Model is most confident that it's an ankle boot. Let's see if it's correct

In [ ]:
test_labels[0]

In [ ]:
class_names[0]

## Experiment Tracking for Tensorflow based Models

In [ ]:
def tf_experiment_tracking(exp_name, run_name, batch_size, epochs, train_loss, train_acc, 
                           val_loss, val_acc, test_loss, test_acc, model_signature):
    mlflow.set_experiment(exp_name)
    with mlflow.start_run(run_name=run_name):
        mlflow.log_param("batch_size", batch_size)
        #mlflow.log_param("learning_rate", learning_rate)
        mlflow.log_param("epochs", epochs)
        mlflow.log_metric("train_loss", train_loss)
        mlflow.log_metric("train_accuracy", train_acc)
        mlflow.log_metric("val_loss", val_loss)
        mlflow.log_metric("val_accuracy", val_acc)
        mlflow.log_metric("test_loss", test_loss)
        mlflow.log_metric("test_accuracy", test_acc)
        mlflow.tensorflow.log_model(model, "mnist", signature=model_signature)
    mlflow.end_run()

In [ ]:
from mlflow.models.signature import infer_signature
model_signature = infer_signature(scaled_test_images, model.predict(scaled_test_images))

In [ ]:
from datetime import datetime
exp_timestamp = datetime.now().strftime("%Y%m%d")
exp_name = "live_mnist_exp_" + exp_timestamp
print(exp_name)

run_timestamp = datetime.now().strftime("%Y%m%d--%H%M%S")
run_name = "live_mnist_exp_run_"+ run_timestamp
print(run_name)

In [ ]:
tf_experiment_tracking(exp_name, run_name, batch_size, epochs, train_loss, train_acc, 
                           val_loss, val_acc, test_loss, test_acc, model_signature)

## Load the model from mlflow run and do prediction

In [ ]:
import mlflow
logged_model = 'runs:/960853ecebfb48ffad16b7f887217945/mnist'

# Load model as a PyFuncModel.
loaded_model = mlflow.pyfunc.load_model(logged_model)
#test_images1 = scaled_test_images.astype('float64')
predictions = loaded_model.predict(scaled_test_images)
predictions[0]

In [ ]:
predictions[4]

In [ ]:
np.argmax(predictions[5])

In [ ]:
test_labels[5]

In [ ]:
class_names[5]

In [ ]:
probability_model = tf.keras.Sequential([model, 
                                         tf.keras.layers.Softmax()])

In [ ]:
predictions = probability_model.predict(scaled_test_images)

In [ ]:
predictions[0]

In [ ]:
np.argmax(predictions[0])

https://www.tensorflow.org/tutorials/keras/classification

## Thank You